In [50]:
import os
import json
import pandas as pd
while os.getcwd() != '/home/jupyter/crisp':
    os.chdir("..")
%pwd

'/home/jupyter/crisp'

In [51]:
from tqdm import tqdm
from sklearn.model_selection import train_test_split
from google.cloud import storage

In [52]:
metadata = pd.read_csv('data/Elijah_mice/mouse_phenotype_data.csv')
index = metadata.corrected
Target = metadata.days
metadata['environment'] = metadata.Cohort.astype(str) + metadata.sex + metadata.group
# # data = pd.read_csv('data/Elijah_mice/geno.txt', delimiter = '\t')
# # data.to_pickle('data/Elijah_mice/geno.pkl')
# data = pd.read_pickle('data/Elijah_mice/geno.pkl')
# data = data.loc[index]
# modal_values = data.mode().iloc[0]
# majority_not_missing = list(modal_values.T[modal_values.T != '-'].index)
# data = data[majority_not_missing]
# modal_values = modal_values[majority_not_missing]
# divergent_snip_data = data != modal_values
# divergent_snip_data = divergent_snip_data.astype(int).astype(float)
# divergent_snip_data.to_pickle('data/Elijah_mice/formatted_data.pkl')

In [56]:
# metadata.query("environment not in ['1Munirradiated','1Funirradiated','2Munirradiated','2Funirradiated','2Fsham irradiated','2Msham irradiated','1Msham irradiated','1Fsham irradiated']")

In [29]:
reduced_modes = [True]

In [30]:
for reduced_mode in reduced_modes:
    data = pd.read_pickle('data/Elijah_mice/formatted_data.pkl')

    if reduced_mode:
        top_1000_variance_columns = list(data.std().sort_values(ascending=False).iloc[:1000].index)
        data = data[top_1000_variance_columns]
        
    # Odhran - you must check this! it might be incorrect if indices have been moved at any point
    data['Target'] = list(Target)
    data['environment'] = list(metadata.environment)
    data = data.query("environment not in ['1Munirradiated','1Funirradiated','2Munirradiated','2Funirradiated','2Fsham irradiated','2Msham irradiated','1Msham irradiated','1Fsham irradiated']")

    storage_client = storage.Client()
    bucket = storage_client.get_bucket('ah21_data')

    if reduced_mode:
        experiment_directory = 'elijah_mouse_experiment_reduced_new_code_v3/'
        print('reduced_mode_on')
    else: 
        experiment_directory = 'elijah_mouse_experiment_new_code_v3/'
        
    for seed in tqdm(range(0,75)):
        seed_directory = 'seed_{}/'.format(str(seed))

        for index, environment in enumerate(data.environment.unique()):
#             colab_directory = 'col_{}_{}/'.format(index, environment.replace(" ", ""))
            colab_directory = 'col_{}/'.format(index)
            env_data = data[data.environment == environment]
            env_data = env_data.drop('environment', axis=1)
            train, test = train_test_split(env_data, test_size=0.2, random_state = seed)

            blob = bucket.blob(experiment_directory+seed_directory+colab_directory+'train/data.csv')
            train.to_csv('elijah_data.csv')
            blob.upload_from_filename('elijah_data.csv')

            blob = bucket.blob(experiment_directory+seed_directory+colab_directory+'test/data.csv')
            test.to_csv('elijah_data.csv')
            blob.upload_from_filename('elijah_data.csv')


  0%|          | 0/58 [00:00<?, ?it/s]

reduced_mode_on


100%|██████████| 58/58 [16:19<00:00, 16.88s/it]


In [47]:
reduced_modes = [False, True]

In [48]:
metadata = pd.read_csv('data/Elijah_mice/mouse_phenotype_data.csv')
index = metadata.corrected
Target = metadata.days
metadata['environment'] = metadata.Cohort.astype(str) + metadata.group

In [49]:
for reduced_mode in reduced_modes:
    data = pd.read_pickle('data/Elijah_mice/formatted_data.pkl')

    if reduced_mode:
        top_1000_variance_columns = list(data.std().sort_values(ascending=False).iloc[:1000].index)
        data = data[top_1000_variance_columns]
        
    # Odhran - you must check this! it might be incorrect if indices have been moved at any point
    data['Target'] = list(Target)
    data['environment'] = list(metadata.environment)
    data = data.query("environment not in ['1unirradiated','2unirradiated','2sham irradiated','1sham irradiated']")

    storage_client = storage.Client()
    bucket = storage_client.get_bucket('ah21_data')

    if reduced_mode:
        experiment_directory = 'final_elijah_mouse_experiment_reduced/'
        print('reduced_mode_on')
    else: 
        experiment_directory = 'final_elijah_mouse_experiment/'
        
    for seed in tqdm(range(0,75)):
        seed_directory = 'seed_{}/'.format(str(seed))

        for index, environment in enumerate(data.environment.unique()):
#             colab_directory = 'col_{}_{}/'.format(index, environment.replace(" ", ""))
            colab_directory = 'col_{}/'.format(index)
            env_data = data[data.environment == environment]
            env_data = env_data.drop('environment', axis=1)
            train, test = train_test_split(env_data, test_size=0.2, random_state = seed)

            blob = bucket.blob(experiment_directory+seed_directory+colab_directory+'train/data.csv')
            train.to_csv('elijah_data.csv')
            blob.upload_from_filename('elijah_data.csv')

            blob = bucket.blob(experiment_directory+seed_directory+colab_directory+'test/data.csv')
            test.to_csv('elijah_data.csv')
            blob.upload_from_filename('elijah_data.csv')


  0%|          | 0/75 [00:00<?, ?it/s]

reduced_mode_on


100%|██████████| 75/75 [09:46<00:00,  7.82s/it]


In [4]:
data = pd.read_pickle('data/Elijah_mice/formatted_data.pkl')

In [20]:
reduced_mode = False

In [21]:
if reduced_mode:
    print('reduced mode running')
    top_1000_variance_columns = list(data.std().sort_values(ascending=False).iloc[:1000].index)
    data = data[top_1000_variance_columns]

In [22]:
# Odhran - you must check this! it might be incorrect if indices have been moved at any point
data['Target'] = list(Target)
data['environment'] = list(metadata.environment)

In [23]:
from google.cloud import storage
storage_client = storage.Client()
bucket = storage_client.get_bucket('ah21_data')
if reduced_mode:
    print('reduced mode running')
    experiment_directory = 'elijah_mouse_experiment_reduced/'
else: 
    experiment_directory = 'elijah_mouse_experiment/'

In [24]:
# need to drop environments here?

In [ ]:
for seed in tqdm(range(0,75)):
    seed_directory = 'seed_{}/'.format(str(seed))
    
    for index, environment in enumerate(data.environment.unique()):
        if 'sham' not in environment:
            if 'unirradiated' not in environment:
                colab_directory = 'col_{}_{}/'.format(index, environment.replace(" ", ""))
                env_data = data[data.environment == environment]
                env_data = env_data.drop('environment', axis=1)
                train, test = train_test_split(env_data, test_size=0.2, random_state = seed)

                blob = bucket.blob(experiment_directory+seed_directory+colab_directory+'train/data.csv')
                train.to_csv('data.csv')
                blob.upload_from_filename('data.csv')

                blob = bucket.blob(experiment_directory+seed_directory+colab_directory+'test/data.csv')
                test.to_csv('data.csv')
                blob.upload_from_filename('data.csv')

 27%|██▋       | 20/75 [1:16:35<3:29:33, 228.60s/it]